In [1]:
COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  local_path = '/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets'
else:
  local_path = r'G:\Mi unidad\ProyectoIA2\generative-spectral-datasets'

import os 
os.chdir(local_path)

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

from models.main import make_autoencoder, make_discriminator
from models.metrics import PSNR, SSIM
from data import load_dataset

input_shape = (512,512,31)

In [3]:
BATCH_SIZE = 3

train_params = dict(
    batch_size = BATCH_SIZE,
    shuffle = True,
    cache = True
)

test_params = dict(
    batch_size = BATCH_SIZE,
    cache = True
)

train_ds , test_ds = load_dataset( train_params = train_params, test_params = test_params)

In [4]:
FACTOR = [ 1 , 1 , 1/2 , 1/2 , 1/4, 1/8]

features= 64

autoencoder = make_autoencoder(input_shape, features=features, factors=FACTOR)
mse_loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(1e-4)

metrics = [PSNR(), SSIM()]
autoencoder.compile( optimizer=optimizer, loss=mse_loss, metrics=metrics)

In [5]:
autoencoder.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 31)]    0         
_________________________________________________________________
encoder (Encoder)            (None, 512, 512, 8)       89208     
_________________________________________________________________
generator (Generator)        (None, 512, 512, 31)      89199     
Total params: 178,407
Trainable params: 177,559
Non-trainable params: 848
_________________________________________________________________


In [ ]:
autoencoder.fit( train_ds , epochs=100, validation_data=test_ds)

Epoch 1/100
9/9 [==============================] - 36s 2s/step - loss: 0.1738 - psnr: 8.2147 - ssim: 0.1926 - val_loss: 0.0190 - val_psnr: 17.6142 - val_ssim: 0.4764
Epoch 2/100
9/9 [==============================] - 11s 1s/step - loss: 0.0594 - psnr: 12.6777 - ssim: 0.2414 - val_loss: 0.0189 - val_psnr: 17.6580 - val_ssim: 0.4731
Epoch 3/100
8/9 [=========================>....] - ETA: 1s - loss: 0.0299 - psnr: 15.6362 - ssim: 0.2797